# Bundling a MONAI Inference Bundle
It is simple to package your MONAI code into a MONAI Bundle. For this tutorial, we will convert a Clara 4.1 model from NGC
- For imports, all we need is the json package

In [1]:
import json

- We'll need to create our bundle directory

In [2]:
# Create app directory structure
APP_ROOT = "/workspace/monai-bundle-demo-covid19-classification"
!mkdir -p $APP_ROOT
!mkdir -p $APP_ROOT/configs
!mkdir -p $APP_ROOT/docs
!mkdir -p $APP_ROOT/models

- At the top of our json, we declare our imports for the bundle
- Next we set a few variables
- Anything that begins with an @ is a variable that is referenced elsewhere in the config file.
- Anything that begins with a $ is evaluabted before being written into the code.

In [3]:
# Common elements
bundle_config = {
    "imports": ["$import glob", "$import os"],
    "device": "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
    "bundle_root": APP_ROOT,
    "model_dir": "$@bundle_root + '/models'",
    "output_dir": "$@bundle_root + '/out'",
    "dataset_dir": "/workspace/datasets",
    "images": "$list(sorted(glob.glob(@dataset_dir + '/CT-Covid-19-August2020/*.nii.gz')))",
}

- Next we download the Clara 4.1 app and extract it.
- In this tutorial, we downloaded and extracted the following:
- https://catalog.ngc.nvidia.com/orgs/nvidia/teams/med/models/clara_pt_covid19_ct_lung_segmentation

In [4]:
CLARA_APP_ROOT = "/workspace/covid19_seg"

In [5]:
!cp $CLARA_APP_ROOT/models/* $APP_ROOT/models/

- We'll run this tutorial on the inference configuration.

In [6]:
# Read CLARA 4.1 Training Config
with open(CLARA_APP_ROOT + "/config/config_inference.json") as f:
    config_infer = json.load(f)

- Here we check the model in the clara app, an define the model in the MONAI Bundle style

In [7]:
# Get model
config_infer["model"]

{'ts_path': '{MMAR_TORCHSCRIPT}'}

In [8]:
# We can have network load directly
bundle_config["network"] = "$torch.jit.load(@model_dir + '/model.ts')"

- Next is preprocessing

In [9]:
# Read Pre-Transforms
pre_transforms = config_infer["pre_transforms"]
for t in pre_transforms:
    print(f"""{t["name"]}({", ".join([f'''{k}={str(v)}''' for k,v in t["args"].items()])})""")

LoadImaged(keys=['image'])
EnsureChannelFirstd(keys=['image'])
Spacingd(keys=['image'], pixdim=[0.8, 0.8, 5], mode=['bilinear'], align_corners=[True])
ScaleIntensityRanged(keys=image, a_min=-1500, a_max=500, b_min=0, b_max=1, clip=True)
CropForegroundd(keys=['image'], source_key=image)
ToTensord(keys=['image'])


In [10]:
# Replace Variables
pre_transforms = config_infer["pre_transforms"]
pre_transforms = {
    "_target_": "Compose",
    "transforms": [
        {"_target_": t["name"], **t["args"]}
        for t in pre_transforms
        if t["name"] != "ToTensord"
    ],
}
pre_transforms

{'_target_': 'Compose',
 'transforms': [{'_target_': 'LoadImaged', 'keys': ['image']},
  {'_target_': 'EnsureChannelFirstd', 'keys': ['image']},
  {'_target_': 'Spacingd',
   'keys': ['image'],
   'pixdim': [0.8, 0.8, 5],
   'mode': ['bilinear'],
   'align_corners': [True]},
  {'_target_': 'ScaleIntensityRanged',
   'keys': 'image',
   'a_min': -1500,
   'a_max': 500,
   'b_min': 0,
   'b_max': 1,
   'clip': True},
  {'_target_': 'CropForegroundd', 'keys': ['image'], 'source_key': 'image'}]}

In [11]:
# Add pre-transforms
bundle_config["pre_transforms"] = pre_transforms

- Next we convert the training dataset and dataloader as well

In [12]:
# Get Dataset and DataLoader
print(config_infer["dataset"])
print(config_infer["dataloader"])

{'name': 'Dataset', 'data_list_file_path': '{DATASET_JSON}', 'data_file_base_dir': '{DATA_ROOT}', 'data_list_key': '{INFER_DATALIST_KEY}', 'args': {'transform': '@pre_transforms'}}
{'name': 'DataLoader', 'args': {'dataset': '@dataset', 'batch_size': 1, 'shuffle': False, 'num_workers': 4}}


- Here we use the first 90% of the images as our training images.
- Notice that we can write code to be evaluated directly in the config

In [13]:
dataset = {
    "_target_": config_infer["dataset"]["name"],
    **config_infer["dataset"]["args"]
}
dataset["data"] = "$[{'image': i} for i in @images]"
dataset["transform"] = "@pre_transforms"
dataloader = {
    "_target_": config_infer["dataloader"]["name"],
    **config_infer["dataloader"]["args"]
}
dataloader["dataset"] = "@dataset"
print(dataset)
print(dataloader)

{'_target_': 'Dataset', 'transform': '@pre_transforms', 'data': "$[{'image': i} for i in @images]"}
{'_target_': 'DataLoader', 'dataset': '@dataset', 'batch_size': 1, 'shuffle': False, 'num_workers': 4}


In [14]:
# Add train dataset and dataloader
bundle_config["dataset"] = dataset
bundle_config["dataloader"] = dataloader

In [15]:
config_infer["inferer"]

{'name': 'SlidingWindowInferer',
 'args': {'roi_size': ['{roi_x}', '{roi_y}', '{roi_z}'],
  'sw_batch_size': 4,
  'overlap': 0.6}}

- Next we set up the inferer to use for training

In [16]:
# Inferer
inferer = json.loads(
    json.dumps(config_infer["inferer"])
    .replace('"{roi_x}"', str(config_infer["roi_x"]))
    .replace('"{roi_y}"', str(config_infer["roi_y"]))
    .replace('"{roi_z}"', str(config_infer["roi_z"]))
)
inferer = {
    "_target_": inferer["name"],
    **inferer["args"],
}
print(inferer)

{'_target_': 'SlidingWindowInferer', 'roi_size': [320, 320, 64], 'sw_batch_size': 4, 'overlap': 0.6}


In [17]:
# Add inferer
bundle_config["inferer"] = inferer

- Next let's take a look at the handlers

In [18]:
[print(handler) for handler in config_infer["handlers"]]

{'name': 'StatsHandler', 'rank': 0, 'args': {'output_transform': 'lambda x: None'}}


[None]

Since the handlers are more complicated, we'll write them one by one.

In [19]:
handlers = [
    {
        "_target_": "StatsHandler",
        "output_transform": "$lambda x: None",
    },
]
[print(handler) for handler in handlers]


{'_target_': 'StatsHandler', 'output_transform': '$lambda x: None'}


[None]

In [20]:
# Add handlers
bundle_config["handlers"] = handlers

- Next we have the postprocessing transforms

In [21]:
config_infer["post_transforms"]

[{'name': 'Activationsd', 'args': {'keys': 'pred', 'softmax': True}},
 {'name': 'Invertd',
  'args': {'keys': 'pred',
   'transform': '@pre_transforms',
   'orig_keys': 'image',
   'meta_keys': 'pred_meta_dict',
   'nearest_interp': False,
   'to_tensor': True,
   'device': 'cuda'}},
 {'name': 'AsDiscreted', 'args': {'keys': 'pred', 'argmax': True}},
 {'name': 'SaveImaged',
  'args': {'keys': 'pred',
   'meta_keys': 'pred_meta_dict',
   'output_dir': '{MMAR_EVAL_OUTPUT_PATH}',
   'resample': False,
   'squeeze_end_dims': True}}]

In [22]:
post_transforms = [
    {
        "_target_": t["name"],
        **{
            k: config_infer["out_classes"]
            if k == "to_onehot"
            else (
                v.replace("{MMAR_EVAL_OUTPUT_PATH}", "@output_dir")
                if isinstance(v, str)
                else v
            )
            for k, v in t["args"].items()
            if k != "n_classes"
        },
    }
    for t in config_infer["post_transforms"]
]
post_transforms = {
    "_target_": "Compose",
    "transforms": post_transforms,
}
post_transforms

{'_target_': 'Compose',
 'transforms': [{'_target_': 'Activationsd', 'keys': 'pred', 'softmax': True},
  {'_target_': 'Invertd',
   'keys': 'pred',
   'transform': '@pre_transforms',
   'orig_keys': 'image',
   'meta_keys': 'pred_meta_dict',
   'nearest_interp': False,
   'to_tensor': True,
   'device': 'cuda'},
  {'_target_': 'AsDiscreted', 'keys': 'pred', 'argmax': True},
  {'_target_': 'SaveImaged',
   'keys': 'pred',
   'meta_keys': 'pred_meta_dict',
   'output_dir': '@output_dir',
   'resample': False,
   'squeeze_end_dims': True}]}

In [23]:
# Add train post_transforms
bundle_config["post_transforms"] = post_transforms

In [24]:
config_infer["evaluator"]

{'name': 'SupervisedEvaluator',
 'args': {'device': 'cuda',
  'val_data_loader': '@dataloader',
  'network': '@model',
  'inferer': '@inferer',
  'postprocessing': '@post_transforms',
  'val_handlers': '@handlers',
  'amp': '{amp}'}}

In [25]:
evaluator = {
    "_target_": config_infer["evaluator"]["name"],
    "device": "@device",
    "val_data_loader": "@dataloader",
    "network": "@network",
    "inferer": "@inferer",
    "postprocessing": "@post_transforms",
    "val_handlers": "@handlers",
    "amp": True,
}
evaluator

{'_target_': 'SupervisedEvaluator',
 'device': '@device',
 'val_data_loader': '@dataloader',
 'network': '@network',
 'inferer': '@inferer',
 'postprocessing': '@post_transforms',
 'val_handlers': '@handlers',
 'amp': True}

In [26]:
bundle_config["evaluator"] = evaluator

- Finally, we add a "training" section
- This includes the code that we will run if we select "training"

In [27]:
bundle_config["evaluating"] = [
    "$setattr(torch.backends.cudnn, 'benchmark', True)",
    "$@evaluator.run()",
]

- This is what the overall bundle_config will look like

In [28]:
bundle_config

{'imports': ['$import glob', '$import os'],
 'device': "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
 'bundle_root': '/workspace/monai-bundle-demo-covid19-lung-segmentation',
 'model_dir': "$@bundle_root + '/models'",
 'output_dir': "$@bundle_root + '/out'",
 'dataset_dir': '/workspace/datasets',
 'images': "$list(sorted(glob.glob(@dataset_dir + '/CT-Covid-19-August2020/*.nii.gz')))",
 'network': "$torch.jit.load(@model_dir + '/model.ts')",
 'pre_transforms': {'_target_': 'Compose',
  'transforms': [{'_target_': 'LoadImaged', 'keys': ['image']},
   {'_target_': 'EnsureChannelFirstd', 'keys': ['image']},
   {'_target_': 'Spacingd',
    'keys': ['image'],
    'pixdim': [0.8, 0.8, 5],
    'mode': ['bilinear'],
    'align_corners': [True]},
   {'_target_': 'ScaleIntensityRanged',
    'keys': 'image',
    'a_min': -1500,
    'a_max': 500,
    'b_min': 0,
    'b_max': 1,
    'clip': True},
   {'_target_': 'CropForegroundd', 'keys': ['image'], 'source_key': 'image'}]},
 '

- Next we'll open up a train.json file and write the config into it.

In [29]:
with open(APP_ROOT + "/configs/inference.json", "w") as f:
    json.dump(bundle_config, f)

- Finally to test our code, we can use `monai.bundle run`
- then we specify `training` which will run the code under the training section that we wrote at the end
- and we specify the config file to be the one that we just wrote into

In [30]:
!python -m monai.bundle run evaluating --config_file $APP_ROOT/configs/inference.json

2022-11-29 06:31:33,459 - INFO - --- input summary of monai.bundle.scripts.run ---
2022-11-29 06:31:33,459 - INFO - > runner_id: 'evaluating'
2022-11-29 06:31:33,459 - INFO - > config_file: '/workspace/monai-bundle-demo-covid19-lung-segmentation/configs/inference.json'
2022-11-29 06:31:33,459 - INFO - ---


2022-11-29 06:31:35,080 - Engine run resuming from iteration 0, epoch 0 until 1 epochs
2022-11-29 06:31:45,591 INFO image_writer.py:193 - writing: /workspace/monai-bundle-demo-covid19-lung-segmentation/out/volume-covid19-A-0000/volume-covid19-A-0000_trans.nii.gz
2022-11-29 06:31:47,149 INFO image_writer.py:193 - writing: /workspace/monai-bundle-demo-covid19-lung-segmentation/out/volume-covid19-A-0001/volume-covid19-A-0001_trans.nii.gz
2022-11-29 06:31:49,792 INFO image_writer.py:193 - writing: /workspace/monai-bundle-demo-covid19-lung-segmentation/out/volume-covid19-A-0002/volume-covid19-A-0002_trans.nii.gz
2022-11-29 06:31:53,329 INFO image_writer.py:193 - writing: /workspace/monai